In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import keras
from keras.layers import Dense,Conv2D, BatchNormalization, Activation,Dropout
from keras.layers import AveragePooling2D, Input, Flatten,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
import keras.backend as K
import cv2
import seaborn as sns
import PIL
import skimage
from keras.models import Model
%matplotlib inline

Using TensorFlow backend.


In [2]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [4]:
def resnet_v2(input_tensor, depth, num_classes=17):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    #inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=input_tensor,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    #y = Flatten()(x)
    #outputs = Dense(num_classes,
     #               activation='softmax',
     #               kernel_initializer='he_normal')(y)

    # Instantiate model.
    #model = Model(inputs=inputs, outputs=outputs)
    return x

In [5]:
import keras.backend as K
import tensorflow as tf
def focal_loss(classes_num, gamma=2., alpha=.25, e=0.1):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''
        import tensorflow as tf
        from tensorflow.python.ops import array_ops
        from keras import backend as K

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_sum(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

In [6]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=92)
L2 = resnet_v2(input_tensor=L2, depth=92)
L3 = resnet_v2(input_tensor=L3, depth=92)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
model.compile(loss=[focal_loss(class_nums)], optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [7]:
test = h5py.File('./round2_test_b_20190211.h5')
test_s1 = np.array(test['sen1'])
test_s2 = np.array(test['sen2'])

In [9]:
pred = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for j in range(5):
    model.load_weights('./model_2/1-5-3input/LCZ_model_FOLD_3_input_%s.h5' % str(j))
    for i in range(4):
        te1 = np.rot90(test_s1,k=i,axes=(1,2))
        te2 = np.rot90(test_s2,k=i,axes=(1,2))
        te3 = np.rot90(test_s3,k=i,axes=(1,2))
        pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te3 = test_s3[:,::-1,:,:]
    pred += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te3 = test_s3[:,:,::-1,:]
    pred += model.predict([te1, te2, te3])

In [10]:
def Preprocessing(x1, x2):
    x1_mean,x1_std = np.mean(x1, axis=(1,2,3)),np.std(x1, axis=(1,2,3))
    x2_mean,x2_std = np.mean(x2, axis=(1,2,3)),np.std(x2, axis=(1,2,3))
    x1 = (x1-x1_mean.reshape(-1,1,1,1))/x1_std.reshape(-1,1,1,1)
    x2 = (x2-x2_mean.reshape(-1,1,1,1))/x2_std.reshape(-1,1,1,1)
    return x1, x2

In [11]:
predp = np.zeros((test_s1.shape[0], 17))
for j in range(5):
    model.load_weights('./model_2/LCZ_model_FOLD_3_input_%s_loss-f-92p.h5' % str(j))
    for i in range(4):
        te1 = np.rot90(test_s1,k=i,axes=(1,2))
        te2 = np.rot90(test_s2,k=i,axes=(1,2))
        te1, te2 = Preprocessing(te1, te2)
        te3 = np.concatenate([te1, te2], axis=-1)
        predp += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te1, te2 = Preprocessing(te1, te2)
    te3 = np.concatenate([te1, te2], axis=-1)
    predp += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te1, te2 = Preprocessing(te1, te2)
    te3 = np.concatenate([te1, te2], axis=-1)
    predp += model.predict([te1, te2, te3])

In [12]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=74)
L2 = resnet_v2(input_tensor=L2, depth=74)
L3 = resnet_v2(input_tensor=L3, depth=74)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [13]:
pred74 = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for j in range(5):
    model.load_weights('./model_2/LCZ_model_FOLD_3_input_%s_loss-f-74.h5' % str(j))
    for i in range(4):
        te1 = np.rot90(test_s1,k=i,axes=(1,2))
        te2 = np.rot90(test_s2,k=i,axes=(1,2))
        te3 = np.rot90(test_s3,k=i,axes=(1,2))
        pred74 += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te3 = test_s3[:,::-1,:,:]
    pred74 += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te3 = test_s3[:,:,::-1,:]
    pred74 += model.predict([te1, te2, te3])

In [14]:
input_shape1 = (32, 32, 8)
input_shape2 = (32, 32, 10)
input_shape3 = (32, 32, 18)
input_1 = Input(shape=input_shape1)
input_2 = Input(shape=input_shape2)
input_3 = Input(shape=input_shape3)
L1 = BatchNormalization()(input_1)
L2 = BatchNormalization()(input_2)
L3 = BatchNormalization()(input_3)
L1 = resnet_v2(input_tensor=L1, depth=56)
L2 = resnet_v2(input_tensor=L2, depth=56)
L3 = resnet_v2(input_tensor=L3, depth=56)

#L1 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L1)
#L2 = Conv2D(16, kernel_size=3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(L2)


L = keras.layers.concatenate([L1, L2, L3])
L = Dense(512, activation='relu')(L)
L = BatchNormalization()(L)
L = Dropout(0.5)(L)
L = Dense(17, activation='softmax', kernel_initializer='he_normal')(L)
model = Model(inputs=[input_1, input_2, input_3], outputs=L)
class_nums = [5068, 24431, 31693,  8651, 16493, 35290,  3269, 39326,
       13584, 11954, 42902,  9514,  9165, 41377,  2392,  7898,
       49359]
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.compile(loss=focal_loss(class_nums), optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [15]:
pred56 = np.zeros((test_s1.shape[0], 17))
test_s3 = np.concatenate([test_s1, test_s2], axis=-1)
for j in range(5):
    model.load_weights('./model_2/LCZ_model_FOLD_3_input_%s_loss-f-56.h5' % str(j))
    for i in range(4):
        te1 = np.rot90(test_s1,k=i,axes=(1,2))
        te2 = np.rot90(test_s2,k=i,axes=(1,2))
        te3 = np.rot90(test_s3,k=i,axes=(1,2))
        pred56 += model.predict([te1, te2, te3])
    te1 = test_s1[:,::-1,:,:]
    te2 = test_s2[:,::-1,:,:]
    te3 = test_s3[:,::-1,:,:]
    pred56 += model.predict([te1, te2, te3])
    te1 = test_s1[:,:,::-1,:]
    te2 = test_s2[:,:,::-1,:]
    te3 = test_s3[:,:,::-1,:]
    pred56 += model.predict([te1, te2, te3])

In [25]:
predss = (pred + pred56 + pred74+predp)/120

In [27]:
result = np.zeros((test_s1.shape[0], 17))
for i ,ind in enumerate(np.argmax(predss,axis=1)):
    result[i][ind] = 1
import pandas as pd
s = pd.DataFrame(result,dtype='int')
s.to_csv('ssssssssssss2.csv', header=None, index=False)